In [2]:
import math
import matplotlib.pyplot as plt
import numpy as np
import scipy
from mpl_toolkits import mplot3d
from scipy import stats

h = 6.6261e-27
vlya = 2.47e15

In [4]:
with open('C:\data\I_PI.txt') as f:
    f = [x.strip() for x in f if x.strip()]
    data = [tuple(map(float,x.split())) for x in f[0:]]
    mean_I = [x[0] for x in data]
    std_I = [x[1] for x in data]
    mean_pI = [x[2] for x in data]
    std_pI = [x[3] for x in data]

with open('C:\data\pmu_m.txt') as f:
    f = [x.strip() for x in f if x.strip()]
    data = [tuple(map(float,x.split())) for x in f[0:]]
    index = [x[0] for x in data]
    mean_p = [x[1] for x in data]
    std_p = [x[2] for x in data]
    mean_n = [x[3] for x in data]
    std_n = [x[4] for x in data]

with open(r'C:\data\b.txt') as f:
    f = [x.strip() for x in f if x.strip()]
    data = [tuple(map(float,x.split())) for x in f[0:]]
    index = [x[0] for x in data]
    b0 = [x[1] for x in data]
    b1 = [x[2] for x in data]
    b2 = [x[3] for x in data]
    b3 = [x[4] for x in data]
    b4 = [x[5] for x in data]
    b5 = [x[6] for x in data]
    
with open(r'C:\data\c.txt') as f:
    f = [x.strip() for x in f if x.strip()]
    data = [tuple(map(float,x.split())) for x in f[0:]]
    index = [x[0] for x in data]
    c2 = [x[1] for x in data]
    c3 = [x[2] for x in data]
    c4 = [x[3] for x in data]
    c5 = [x[4] for x in data]
    c6 = [x[5] for x in data]
    c7 = [x[6] for x in data]
    
U = []
nH = []

for i in range(len(index)):
    if index[i]%10 == 0:
        nH_i = 1e-6
    if index[i]%10 == 1:
        nH_i = 5e-6
    if index[i]%10 == 2:
        nH_i = 1e-5
    if index[i]%10 == 3:
        nH_i = 5e-5    
    if index[i]%10 == 4:
        nH_i = 1e-4
    if index[i]%10 == 5:
        nH_i = 5e-4
    if index[i]%10 == 6:
        nH_i = 1e-3
    if index[i]%10 == 7:
        nH_i = 5e-3
    if index[i]%10 == 8:
        nH_i = 1e-2
    nH.append(float(nH_i))
    
    if int(index[i]/10) == 0:
        u_i = 1e7
    if int(index[i]/10) == 1:
        u_i = 3e7
    if int(index[i]/10) == 2:
        u_i = 7e7
    if int(index[i]/10) == 3:
        u_i = 1e8
    if int(index[i]/10) == 4:
        u_i = 3e8
    if int(index[i]/10) == 5:
        u_i = 7e8
    if int(index[i]/10) == 6:
        u_i = 1e9
    if int(index[i]/10) == 7:
        u_i = 3e9
    if int(index[i]/10) == 8:
        u_i = 7e9
    if int(index[i]/10) == 9:
        u_i = 1e10
    U.append(int(u_i))

In [3]:
def P0(x):
    return 1
def P1(x):
    return x
def P2(x):
    return (3*x**2)/2. - 1/2.
def P3(x):
    return (5*x**3)/2. - 3*x/2.
def P4(x):
    return (35*x**4)/8. - (15*x**2)/4. + 3/8.
def P5(x):
    return (63*x**5)/8. - (35*x**3)/4. + 15*x/8.

def P22(x):
    return 3*(1-x**2)
def P23(x):
    return 15*x*(1-x**2)
def P24(x):
    return 15/2 * (7*x**2-1)*(1-x**2)
def P25(x):
    return 1/4*(11*x*P24(x) - 7*P23(x))
def P26(x):
    return 1/5*(13*x*P25(x) - 8*P24(x))
def P27(x):
    return 1/6*(15*x*P26(x) - 9*P25(x))

In [8]:
def fit_2d(x, y, z):
    if x < 3e7:
        U_index = 0
    if 3e7 <= x < 7e7:
        U_index = 1
    if 7e7 <= x < 1e8:
        U_index = 2
    if 1e8 <= x <= 3e8:
        U_index = 3
    if 3e8 <= x <= 7e8:
        U_index = 4
    if 7e8 <= x <= 1e9:
        U_index = 5
    if 1e9 <= x <= 3e9:
        U_index = 6
    if 3e9 <= x <= 7e9:
        U_index = 7
    if x >= 7e9:
        U_index = 8
        
    if y < 5e-6:
        nH_index = 0
    if 5e-6 <= y < 1e-5:
        nH_index = 1
    if 1e-5 <= y < 5e-5:
        nH_index = 2
    if 5e-5 <= y < 1e-4:
        nH_index = 3
    if 1e-4 <= y < 5e-4:
        nH_index = 4
    if 5e-4 <= y < 1e-3:
        nH_index = 5
    if 1e-3 <= y < 5e-3:
        nH_index = 6
    if y >= 5e-3:
        nH_index = 7
        
    index_jk = float(str(U_index)+str(nH_index))
    index_jk1 = float(str(U_index)+str(nH_index+1))
    index_j1k = float(str(U_index+1)+str(nH_index))
    index_j1k1 = float(str(U_index+1)+str(nH_index+1))
    
    jk = index.index(index_jk)
    jk1 = index.index(index_jk1)
    j1k = index.index(index_j1k)
    j1k1 = index.index(index_j1k1)
    
    f_A = z[jk] + (x - U[jk]) / (U[j1k] - U[jk]) * (z[j1k] - z[jk])
    f_B = z[jk1] + (x - U[jk]) / (U[j1k] - U[jk]) * (z[j1k1] - z[jk1])
    f_xy = f_A + (y - nH[jk]) / (nH[jk1] - nH[jk]) * (f_B - f_A)
    
    return f_xy

In [11]:
class I_pI:
    def __init__(self, z, theta, Ui, nHi):
        self.z = z
        self.theta = theta
        self.mu = np.cos(theta)
        self.Ui = Ui
        self.nHi = nHi
    
    def I(self):
        b0_fit = fit_2d(self.Ui, self.nHi, b0)
        b1_fit = fit_2d(self.Ui, self.nHi, b1)
        b2_fit = fit_2d(self.Ui, self.nHi, b2)
        b3_fit = fit_2d(self.Ui, self.nHi, b3)
        b4_fit = fit_2d(self.Ui, self.nHi, b4)
        b5_fit = fit_2d(self.Ui, self.nHi, b5)
        x = self.mu
        pmu_fit = b0_fit + b1_fit * P1(x) + b2_fit * P2(x) + b3_fit * P3(x) + b4_fit * P4(x) + b5_fit * P5(x)
        
        n_fit = fit_2d(self.Ui, self.nHi, mean_n)
        
        vlya_z = vlya / (1 + self.z)
        
        return h * vlya_z * n_fit * pmu_fit / (2 * np.pi * np.abs(np.cos(self.theta)) * (1 + self.z)**3)
    
    def pI(self):
        c2_fit = fit_2d(self.Ui, self.nHi, c2)
        c3_fit = fit_2d(self.Ui, self.nHi, c3)
        c4_fit = fit_2d(self.Ui, self.nHi, c4)
        c5_fit = fit_2d(self.Ui, self.nHi, c5)
        c6_fit = fit_2d(self.Ui, self.nHi, c6)
        c7_fit = fit_2d(self.Ui, self.nHi, c7)
        x = self.mu
        pImu_fit = c2_fit * P22(x) + c3_fit * P23(x) + c4_fit * P24(x)+ c5_fit * P25(x) + c6_fit * P26(x) + c7_fit * P27(x)
        
        n_fit = fit_2d(self.Ui, self.nHi, mean_n)
        
        vlya_z = vlya / (1 + self.z)
        
        return h * vlya_z * n_fit * pImu_fit / (2 * np.pi * np.abs(np.cos(self.theta)) * (1 + self.z)**3)

In [13]:
I_list = []
pI_list = []

for i in range(len(index)):
    ipi = I_pI(8, 3*np.pi/4, U[i], nH[i])
    I_i = ipi.I()
    I_list.append(float(I_i))
    pI_i = ipi.pI()
    pI_list.append(float(pI_i))
    
I_diff = [a-b for a, b in zip(mean_I, I_list)]
I_diff_per = [a/b for a, b in zip(I_diff, mean_I)]
print(max(I_diff_per), min(I_diff_per), np.mean(I_diff_per))

pI_diff = [a-b for a, b in zip(mean_pI, pI_list)]
pI_diff_per = [a/b for a, b in zip(pI_diff, mean_pI)]
print(max(pI_diff_per), min(pI_diff_per), np.mean(pI_diff_per))

6.106527458115584e-16 -7.930181586806359e-16 -7.019118417261838e-18
6.360035193601918e-16 -9.352262193971585e-16 -3.209584030454785e-17
